In [ ]:
import os
import re
import sys
import glob
import numpy as np
from scipy.signal import welch
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator, NullLocator, FixedFormatter
fontsize = 9
lw = 0.75
matplotlib.rc('font', **{'family': 'Arial', 'size': fontsize})
matplotlib.rc('axes', **{'linewidth': 0.75, 'labelsize': fontsize})
matplotlib.rc('xtick', **{'labelsize': fontsize})
matplotlib.rc('ytick', **{'labelsize': fontsize})
matplotlib.rc('xtick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.minor', **{'width': lw, 'size':1.5})
OU_tau = 20e-3 # [s]
dB     = 20

Figure out what data needs to be loaded:

In [ ]:
network = 'IEEE39_CIG'
if network in ('SM_with_load','SM_with_line_and_load','SM_with_line_and_loads','SMs_with_line_and_loads'):
    F0 = 50 # [Hz]
    PF_net_name = network
    load_type = 'static_load_const_Z'
    expt_name = load_type
    if 'loads' in network:
        expt_name = os.path.join(expt_name, 'LD1_LD2')
    fmin, fmax, steps_per_decade = -6, 2, 1000
    outfile = 'TF_{}-{}'.format(PF_net_name, expt_name.replace(os.path.sep,'-'))
elif network == 'IEEE39':
    F0 = 60 # [Hz]
    PF_net_name = 'IEEE39_stoch'
    condition = 'default'
    load_type = 'static_load_const_Z'
    load_name = 'Load_03_Load_21'
    expt_name = os.path.join(condition,load_type,load_name)
    fmin, fmax, steps_per_decade = -6, 2, 1000
    outfile = 'TF_{}-{}-{}-{}'.format(PF_net_name, condition, load_type, load_name)
elif network == 'IEEE39_CIG':
    F0 = 60 # [Hz]
    PF_net_name = 'IEEE39_stoch_CIG'
    buses = [8,14]
    Ta = 0.1
    load_type = 'static_load_const_Z'
    load_name = 'Load_03'
    if len(buses) == 0:
        condition = 'NO_CIG'
#         condition = 'G_07_replacement'
        expt_name = os.path.join(condition,load_type,load_name)
    else:
        condition = 'Bus_' + '_'.join(list(map(lambda s: f'{s:02d}',buses)))
        expt_name = os.path.join(condition,f'Ta={Ta}',load_type,load_name)
    fmin, fmax, steps_per_decade = -6, 2, 100
    outfile = 'TF_{}-{}-{}-{}'.format(PF_net_name, condition, load_type, load_name)
elif network == 'Sardinia':
    F0 = 50 # [Hz]
    static_load = True
    if static_load:
        negative_load = False
        if negative_load:
            PF_net_name = 'V2020_Rete_Sardegna_2021_06_03cr_stoch'
        else:
            PF_net_name = 'V2020_Rete_Sard_2021_06_03cr_mod_loads'
        condition = 'default'
#         condition = 'FSACTI0201GGR3_double_H'
        load_name = 'EqX_NARCDI1501TRB_____LOAD____'
        dP = 0.1
        subdir = os.path.join('negative_load' if negative_load else 'positive_load', f'dP_{dP:g}')
    else:
        PF_net_name = 'V2020_Rete_Sardegna_2021_06_03cr_stoch'
        condition = 'default'
        load_name = 'EqX_BNFC_I0601TRR_____LOAD____'
        dP = 0.01
        subdir = os.path.join(f'dP_{dP}')
    expt_name = os.path.join(condition, load_name, subdir)
    fmin, fmax, steps_per_decade = -6, 4, 100
    outfile = 'TF_{}-{}-{}'.format(PF_net_name, load_name, subdir.split(os.path.sep)[-1])
else:
    raise Exception(f'Unknown network: "{network}"')
folder = os.path.join('..','..','modal_analysis',network,expt_name)
# folder = os.path.join('/','Users','daniele','Google Drive','My Drive','PoliMi',
#                       'modal_analysis',network,expt_name)
if not os.path.isdir(folder):
    raise Exception(f'{folder}: no such folder')
outfile = os.path.join(folder, outfile)
AC_data_file = os.path.join(folder, f'{PF_net_name}_AC_TF_{fmin:.1f}_{fmax:.1f}_{steps_per_decade}.npz')
AC_data = np.load(AC_data_file, allow_pickle=True)
M = AC_data['Mtot'].item()
E = AC_data['Etot'].item()
try:
    bus_equiv_terms = AC_data['bus_equiv_terms'].item()
except:
    pass

#### Transient simulations
First, we load the data corresponding to the requested variable.

Currently, this must be one of the following:
  1. `s:xspeed` for synchronous machines;
  1. `m:ur`, `m:ui`, `m:fe` for buses;
  1. `U`, `theta` and `omega`, for buses, with the addition that these variables are not directly available, but rather computed offline.

In [ ]:
discard = 300 # [s]
var_name = 'm:fe'
# dev is meaningful only if var_name is 's:xspeed', in which case it can be either 'gen' or 'static_gen'
dev = 'gen'
tran_data_files = sorted(glob.glob(os.path.join(folder,f'{PF_net_name}_tran_*.npz')))
n_tran_data_files = len(tran_data_files)
print('Found {} data file{}.'.format(n_tran_data_files, 's' if n_tran_data_files>1 else ''))

if n_tran_data_files > 0:
    tran_blobs = [np.load(f, allow_pickle=True) for f in tran_data_files]
    time = [blob['time'] for blob in tran_blobs]
    dt = time[0][1] - time[0][0]
    ref_SM_name = tran_blobs[0]['ref_SMs'][0]

    if var_name not in ('s:xspeed', 'm:ur', 'm:ui', 'm:fe', 'U', 'theta', 'omega'):
        raise Exception(f'Do not know how to deal with variable "{var_name}"')

    outfile += '-' + var_name.replace(':','_')
    if var_name == 'U':
        X = [np.abs(blob['data'].item()['bus']['m:ur'] + \
                    1j * blob['data'].item()['bus']['m:ui']) for blob in tran_blobs]
        tran_names = tran_blobs[0]['device_names'].item()['bus']
    elif var_name == 'theta':
        X = [np.angle(blob['data'].item()['bus']['m:ur'] + \
                      1j * blob['data'].item()['bus']['m:ui']) for blob in tran_blobs]
        tran_names = tran_blobs[0]['device_names'].item()['bus']
    elif var_name == 'omega':
        θ = [np.angle(blob['data'].item()['bus']['m:ur'] + \
                      1j * blob['data'].item()['bus']['m:ui']) for blob in tran_blobs]
        X = [(th[1:] - th[:-1]) / dt for th in θ]
        time = [t[:-1] for t in time]
        tran_names = tran_blobs[0]['device_names'].item()['bus']
    else:
        if var_name != 's:xspeed':
            dev = 'bus'
        X = [blob['data'].item()[dev][var_name] for blob in tran_blobs]
        tran_names = tran_blobs[0]['device_names'].item()[dev]
    if X[0].ndim == 1:
        X = [np.reshape(x, (-1,1)) for x in X]
    X = np.concatenate([x[t>discard,:] for t,x in zip(time, X)], axis=0)
    tran_time = np.arange(X.shape[0]) * dt
else:
    tran_time,X = None, None
    if network == 'Sardinia':
        ref_SM_name = 'BNFC_I0601GGR1____GEN_____'
    elif 'IEEE39' in network:
        ref_SM_name = 'G 02'
        tran_names = [f'G {i+1:02d}' for i in range(10)]

Then, we compute the PSDs of the data:

In [ ]:
if n_tran_data_files > 0:
    if var_name in ('m:ur', 'm:ui', 'theta', 'omega'):
        ΔX = X
    elif var_name in ('s:xspeed', 'U', 'm:fe'):
        ΔX = X-1
    else:
        raise Exception('Do not know how to compute ΔX')
    window_dur = 60 * 15
    window = window_dur / dt
    onesided = True
    tran_freq,tran_Pxx = welch(ΔX, 1/dt, window='boxcar', nperseg=window, noverlap=window/2,
                               nfft=window, return_onesided=True, scaling='density', axis=0)
    tran_Pxx /= 2
    tran_freq,tran_Pxx = tran_freq[1:],tran_Pxx[1:,:]
    tran_mag = {name: 10*np.log10(tran_Pxx[:,i]) for i,name in enumerate(tran_names)}

### Transfer functions computed analytically

Depending on the variable needed, we have to either select directly the required transfer function (this is the case if the variable is among those that appear in the Jacobian matrix of PowerFactory) or to build it using those that are available.

In [ ]:
def find_var_name(all_names, obj_name, var_name):
    full_names = [name for name in all_names if re.search(obj_name+'\.', name) is not None \
                  and re.search('\.{}$'.format(var_name), name) is not None]
    if len(full_names) == 0:
        return None
    if len(full_names) == 1:
        return np.where(all_names == full_names[0])[0][0]
    print(full_names)
    raise Exception('{} instances of variables containing `{}` and `{}`'.format(len(full_names), obj_name, var_name))

In [ ]:
SM_names = list(AC_data['SM_names'])
bus_names = list(AC_data['bus_names'])
AC_freq = AC_data['F']
AC_TF, AC_mag = {}, {}
PF = AC_data['PF'].item()
for name in tran_names:
    if var_name == 's:xspeed':
        idx = find_var_name(AC_data['var_names'], name, 'speed')
        AC_TF[name] = np.sqrt(np.sum(AC_data['TF'][:,:,idx]**2, axis=0))
        AC_mag[name] = dB*np.log10(np.abs(AC_TF[name]))
    elif var_name in ('m:ur','m:ui'):
        u = var_name.split(':')[1]
        idx = find_var_name(AC_data['var_names'], name, u)
        if idx is None:
            for equiv_term in bus_equiv_terms[name]:
                idx = find_var_name(AC_data['var_names'], equiv_term, u)
                if idx is not None:
                    break
        AC_TF[name] = np.sqrt(np.sum(AC_data['TF'][:,:,idx]**2, axis=0))
        AC_mag[name] = dB*np.log10(np.abs(AC_TF[name]))
    elif var_name == 'U':
        raise NotImplementedError('not implemented yet')
        if name+'.ur' not in AC_data['var_names']:
            for equiv_term in bus_equiv_terms[name]:
                if equiv_term+'.ur' in AC_data['var_names']:
                    idx_ur, = np.where(AC_data['var_names'] == equiv_term+'.ur')
                    idx_ui, = np.where(AC_data['var_names'] == equiv_term+'.ui')
                    ur,ui = PF['buses'][equiv_term]['ur'], PF['buses'][equiv_term]['ui']
                    break
        else:
            idx_ur, = np.where(AC_data['var_names'] == name+'.ur')
            idx_ui, = np.where(AC_data['var_names'] == name+'.ui')
            ur,ui = PF['buses'][name]['ur'], PF['buses'][name]['ui']
        coeff_ur,coeff_ui = np.array([ur,ui]) / np.sqrt(ur**2+ui**2)
        TF = coeff_ur*AC_data['TF'][:,:,idx_ur[0]] + coeff_ui*AC_data['TF'][:,:,idx_ui[0]]
        AC_TF[name] = np.sqrt(np.sum(TF**2, axis=0))
        AC_mag[name] = dB * np.log10(np.abs(AC_TF[name]))
    elif var_name in ('m:fe','theta','omega'):
        idx_ur = find_var_name(AC_data['var_names'], name, 'ur')
        if idx_ur is not None:
            idx_ui = find_var_name(AC_data['var_names'], name, 'ui')
            ur,ui = PF['buses'][name]['ur'], PF['buses'][name]['ui']
        else:
            for equiv_term in bus_equiv_terms[name]:
                idx_ur = find_var_name(AC_data['var_names'], equiv_term, 'ur')
                if idx_ur is not None:
                    idx_ui = find_var_name(AC_data['var_names'], equiv_term, 'ui')
                    ur,ui = PF['buses'][equiv_term]['ur'], PF['buses'][equiv_term]['ui']
                    break
        if ur == 0:
            print('{}: ur,ui = ({:g},{:g})'.format(name, ur, ui))
            continue
        coeff_ur = -ui/ur**2/(1+(ui/ur)**2)
        coeff_ui = 1/(ur*(1+(ui/ur)**2))
        TF = coeff_ur*AC_data['TF'][:,:,idx_ur] + coeff_ui*AC_data['TF'][:,:,idx_ui]
        if var_name in ('m:fe','omega'):
            TF *= 1j*2*np.pi*AC_freq # Δω = jωΔθ
            if var_name == 'm:fe':
                ref_SM_idx = find_var_name(AC_data['var_names'], ref_SM_name, 'speed')
                TF /= 2*np.pi*F0 # !!! scaling factor !!!
                TF += AC_data['TF'][:,:,ref_SM_idx]
        AC_TF[name] = np.sqrt(np.sum(TF**2, axis=0))
        AC_mag[name] = dB * np.log10(np.abs(AC_TF[name]))

#### Save the data

In [ ]:
from scipy.io import savemat
out = {}
out['F'] = AC_freq
out['TF'] = np.array(list(AC_TF.values()))
out['mag'] = np.array(list(AC_mag.values()))
out['var_names'] = list(AC_TF.keys())
for key in ('Htot','Etot','Mtot','H','S','P','Q','SM_names'):
    out[key] = AC_data[key]
savemat(outfile + '.mat', out)

#### Plot the results

In [ ]:
if n_tran_data_files > 0:
    assert np.all([k in tran_mag.keys() for k in AC_mag.keys()]) #and np.all([k in AC_mag.keys() for k in tran_mag.keys()])

try:
    device_names = list(AC_mag.keys()) if len(AC_mag) > len(tran_mag) else list(tran_mag.keys())
except:
    device_names = list(AC_mag.keys())
if len(device_names) in (9,10):
    rows,cols = 2,5
    w,h = 3,2.5
elif len(device_names) == 18:
    rows,cols = 3,6
    w,h = 3,2.5
elif len(device_names) == 39:
    rows,cols = 13,3
    w,h = 3,2.5
elif len(device_names) == 43:
    rows,cols = 11,4
    w,h = 3,2.5
elif len(device_names) == 2:
    rows,cols = 1,2
    w,h = 4,3
elif len(device_names) == 1:
    rows,cols = 1,1
    w,h = 5,4

# pretty-plot
pp = False
if pp:
    if len(device_names) == 1:
        w,h = 10/2.54, 6/2.54
    else:
        rows,cols = 2,2
        w,h = 7.5/2.54, 5/2.54

fig,ax = plt.subplots(rows, cols, figsize=(cols*w, rows*h), sharex=True, sharey=True, squeeze=False)
ticks = np.logspace(-3, 4, 8)
red,green,magenta,orange = [0.75,0,0], [0,.75,0], [.75,0,.75], [1,.5,0]
col = [green,magenta]
cmap = plt.get_cmap('tab10', len(AC_data))
lw = 1
if pp and len(device_names) == 18:
    devices_to_plot = ['FSACTI0201GGR3____GEN_____', 'BNFC_I0601GGR1____GEN_____',
                       'SULCTI0202GGR2____GEN_____', 'FL1CZI0101GGR1____GEN_____']
else:
    devices_to_plot = device_names[:rows*cols]

if n_tran_data_files > 0:
    tran_idx = (tran_freq >= ticks[0]) & (tran_freq <= ticks[-1])
AC_idx = (AC_freq >= ticks[0]) & (AC_freq <= ticks[-1])

def bounds(x, m, M):
    mm,MM = x.min(),x.max()
    if not np.isinf(mm) and not np.isnan(mm) and mm < m and mm > -400:
        m = mm
    if not np.isinf(MM) and not np.isnan(MM) and MM > M:
        M = MM
    return m,M
ym,yM = 0,-200

for k,name in enumerate(devices_to_plot):
    i,j = k//cols, k%cols
    if n_tran_data_files > 0 and name in tran_mag:
        ### TRAN
        y = tran_mag[name]
        ym,yM = bounds(y[tran_idx], ym, yM)
        ax[i,j].semilogx(tran_freq, y, 'k', lw=lw, label=r'TRAN: E = {:.2f} GW$\cdot$s'.format(E*1e-3))
    ### AC
    if name in AC_mag:
        y = AC_mag[name]
        ym,yM = bounds(y[AC_idx], ym, yM)
        ax[i,j].semilogx(AC_freq, AC_mag[name], color=red, lw=1.5, label='AC')
    ax[i,j].set_title(name.split('___')[0], fontsize=8)

for i in range(rows):
    for j in range(cols):
        ax[i,j].plot(1/(2*np.pi*OU_tau)+np.zeros(2), [ym,yM], ':', color=orange, lw=2, label='OU cutoff')
        ax[i,j].xaxis.set_major_locator(FixedLocator(ticks))
        ax[i,j].xaxis.set_minor_locator(NullLocator())
        ax[i,j].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in ticks]))        
ax[0,0].set_xlim(ticks[[0,-1]])
ax[0,0].set_ylim([ym,yM])
ax[0,0].legend(loc='lower left', frameon=False, fontsize=6)
for a in ax[-1,:]:
    a.set_xlabel('Frequency [Hz]')
for a in ax[:,0]:
    a.set_ylabel(f'PSD [dB{dB}]')
sns.despine()
fig.tight_layout()
plt.savefig(outfile + '.pdf')

In [ ]:
if network == 'Sardinia' and var_name == 's:xspeed':
    N_gen = len(device_names)
    groups = ['AS','BMA','BMS','BN','CO','FL','FS','OZ','SE','SL','SU','TI']
    N_groups = len(groups)
    cmap = sns.color_palette('tab20', n_colors=N_groups)
    fig,ax = plt.subplots(1, 1, figsize=(10/2.54,6/2.54))
    x = [0.0012, 0.03]
    y = np.linspace(-140, -195, 9)
    for k,(key,val) in enumerate(AC_mag.items()):
        name = key.split('__')[0]
        if name[:2] == 'BM':
            col = cmap[groups.index(name[:3])]
        else:
            col = cmap[groups.index(name[:2])]
        ax.semilogx(AC_freq, val, color=col, lw=0.75, label=name)
        i,j = k//9,k%9
        ax.text(x[i], y[j], name, fontsize=6, color=col)
    ax.xaxis.set_major_locator(FixedLocator(ticks))
    ax.xaxis.set_minor_locator(NullLocator())
    ax.xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in ticks]))
    ax.set_xlim(ticks[[0,-1]])
    ax.set_xlabel('Frequency [Hz]')
    ax.set_ylabel(f'PSD [dB{dB}]')
    sns.despine()
    fig.tight_layout()
    plt.savefig(outfile + '_all_gen.pdf')